# Group Splitter (ZenML)

Train the `GroupSplitter` on real preprocessed time-group data:
- Load preprocessed splitter groups from .npy files in the data folder
- Train the `GroupSplitter` via a ZenML pipeline with Optuna hyperparameter tuning
- Reload artifacts and render evaluation plots (confusion matrices, ROC curves, etc.)


In [ ]:
from pathlib import Path
from pioneerml.zenml import load_step_output
from pioneerml.zenml import utils as zenml_utils
from pioneerml.zenml.pipelines.training import group_splitter_optuna_pipeline
from pioneerml.data import NODE_LABEL_TO_NAME, NUM_NODE_CLASSES

PROJECT_ROOT = zenml_utils.find_project_root()
zenml_client = zenml_utils.setup_zenml_for_notebook(root_path=PROJECT_ROOT, use_in_memory=True)
print(f"ZenML ready with stack: {zenml_client.active_stack_model.name}")


Initializing the ZenML global configuration version to 0.92.0
Creating database tables
Creating default project 'default' ...
Creating default stack...
Setting the global active project to 'default'.
Setting the global active stack to default.
Using ZenML repository root: /home/jack/python_projects/pioneerML
Ensure this is the top-level of your repo (.zen must live here).
ZenML ready with stack: default


In [ ]:
# Configure the pipeline
# Use absolute path based on project root
file_pattern = str(Path(PROJECT_ROOT) / 'data' / 'mainTimeGroups_*.npy')
run = group_splitter_optuna_pipeline.with_options(enable_cache=False)(
    build_datamodule_params={
        # Data loading parameters
        'file_pattern': file_pattern,
        'max_files': 20,
        'limit_groups': 1000000,
        'min_hits': 1,
        # Datamodule parameters
        'use_group_probs': False,  # Set to True if you have classifier probabilities
        'batch_size': 8,
        'num_workers': 0,
        'val_split': 0.15,
        'seed': 42,
    },
    run_hparam_search_params={
        'n_trials': 25,
        'max_epochs': 20,
        'limit_train_batches': 0.8,
        'limit_val_batches': 1.0,
    },
    train_best_model_params={
        'max_epochs': 50,
        'early_stopping': True,
        'early_stopping_patience': 6,
        'early_stopping_monitor': 'val_loss',
        'early_stopping_mode': 'min',
    },
)
print(f"Run name: {run.name}")
print(f"Run status: {run.status}")


Initiating a new run for the pipeline: group_splitter_optuna_pipeline.
Registered new pipeline: group_splitter_optuna_pipeline.
Caching is disabled by default for group_splitter_optuna_pipeline.
Using user: default
Using stack: default
  artifact_store: default
  deployer: default
  orchestrator: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml login --local.
Step load_splitter_data has started.
Failed to run step load_splitter_data: No files matched pattern 'data/mainTimeGroups_*.npy'
Step load_splitter_data failed.
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/jack/miniconda3/envs/pioneerml/lib/python3.11/site-packages/zenml/orchestrators/local/local_orchestrator.py", line 160, in submit_pipeline
    self.run_step(step=step)
  File "/home/jack/miniconda3/envs/pioneerml/lib/python3.11/site-packages/zenml/orchestrators/local/local_orchestrator.py", line 160, in submit_pipeline
    sel

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│    1 # Configure the pipeline                                                                    │
│    2 # Update file_pattern to point to your data files                                           │
│ ❱  3 run = group_splitter_optuna_pipeline.with_options(enable_cache=False)(                      │
│    4 │   load_data_params={                                                                      │
│    5 │   │   'file_pattern': 'data/mainTimeGroups_*.npy',  # Update this path as needed          │
│    6 │   │   'max_files': 20,                                                                    │
│                                                                                                  │
│ /home/jack/miniconda3/envs/pioneerml/lib/python3.11/site-packages/zenml/pipelines/pipeline_defin │
│ ition.py:1628 in __call__                                                                        │
│                                                                                                  │
│   1625 │   │   │   return self.entrypoint(*args, **kwargs)  # type: ignore[no-any-return]        │
│   1626 │   │                                                                                     │
│   1627 │   │   self.prepare(*args, **kwargs)                                                     │
│ ❱ 1628 │   │   return self._run()                                                                │
│   1629 │                                                                                         │
│   1630 │   def _call_entrypoint(self, *args: Any, **kwargs: Any) -> Any:                         │
│   1631 │   │   """Calls the pipeline entrypoint function with the given arguments.               │
│                                                                                                  │
│ /home/jack/miniconda3/envs/pioneerml/lib/python3.11/site-packages/zenml/pipelines/pipeline_defin │
│ ition.py:1102 in _run                                                                            │
│                                                                                                  │
│   1099 │   │   │   │   │   │   │   "`zenml login --local`."                                      │
│   1100 │   │   │   │   │   │   )                                                                 │
│   1101 │   │   │   │                                                                             │
│ ❱ 1102 │   │   │   │   submit_pipeline(                                                          │
│   1103 │   │   │   │   │   snapshot=snapshot, stack=stack, placeholder_run=run                   │
│   1104 │   │   │   │   )                                                                         │
│   1105                                                                                           │
│                                                                                                  │
│ /home/jack/miniconda3/envs/pioneerml/lib/python3.11/site-packages/zenml/execution/pipeline/utils │
│ .py:115 in submit_pipeline                                                                       │
│                                                                                                  │
│   112 │   │   │   │   # the run as failed if it's still in an unfinished state.                  │
│   113 │   │   │   │   publish_failed_pipeline_run(placeholder_run.id)                            │
│   114 │   │   │                                                                                  │
│ ❱ 115 │   │   │   raise e                                                                        │
│   116                                                      

In [ ]:
trained_module = load_step_output(run, "train_best_splitter")
datamodule = load_step_output(run, "build_splitter_datamodule")
predictions = load_step_output(run, "collect_splitter_predictions", index=0)
targets = load_step_output(run, "collect_splitter_predictions", index=1)
best_params = load_step_output(run, "run_splitter_hparam_search")

if trained_module is None or datamodule is None:
    raise RuntimeError("Could not load artifacts from the optuna pipeline run.")

datamodule.setup(stage="fit")
trained_module.eval()
device = next(trained_module.parameters()).device
val_size = len(datamodule.val_dataset) if datamodule.val_dataset is not None else len(datamodule.train_dataset)
print(f"Loaded module on {device}; validation samples: {val_size}")
print("Best params from Optuna:", best_params)
print("Epochs actually run:", getattr(trained_module, "final_epochs_run", None))


In [ ]:
# Class names for splitter (per-hit classification)
class_names = [NODE_LABEL_TO_NAME[i] for i in range(NUM_NODE_CLASSES)]
print("Class names (index-aligned):", class_names)


In [ ]:
from pioneerml.evaluation.plots import (
    plot_loss_curves,
    plot_multilabel_confusion_matrix,
    plot_precision_recall_curves,
    plot_roc_curves,
)

plot_loss_curves(trained_module, title="Group splitter: loss", show=True)
plot_multilabel_confusion_matrix(
    predictions=predictions,
    targets=targets,
    class_names=class_names,
    threshold=0.5,
    normalize=True,
    show=True,
)
plot_roc_curves(predictions, targets, class_names=class_names, show=True)
plot_precision_recall_curves(predictions, targets, class_names=class_names, show=True)


## Save the Trained Model

Save the trained model and metadata for later use.


In [ ]:
import torch
import json
from datetime import datetime

# Create checkpoints directory
checkpoints_dir = Path(PROJECT_ROOT) / "artifacts" / "checkpoints" / "group_splitter"
checkpoints_dir.mkdir(parents=True, exist_ok=True)

# Generate a timestamped filename
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_filename = f"group_splitter_{timestamp}.pt"
metadata_filename = f"group_splitter_{timestamp}_metadata.json"

# Extract the underlying model from the Lightning module
model = trained_module.model
model.eval()

# Save model state_dict
model_path = checkpoints_dir / model_filename
torch.save(model.state_dict(), model_path)
print(f"Saved model state_dict to: {model_path}")

# Save metadata (hyperparameters, training info, etc.)
metadata = {
    "model_type": "GroupSplitter",
    "timestamp": timestamp,
    "run_name": run.name,
    "best_hyperparameters": best_params,
    "training_config": getattr(trained_module, "training_config", {}),
    "epochs_run": getattr(trained_module, "final_epochs_run", None),
    "dataset_info": {
        "train_size": len(datamodule.train_dataset) if datamodule.train_dataset else 0,
        "val_size": len(datamodule.val_dataset) if datamodule.val_dataset else 0,
        "num_classes": NUM_NODE_CLASSES,
        "class_names": class_names,
        "use_group_probs": getattr(datamodule, "use_group_probs", False),
    },
    "model_architecture": {
        "num_classes": NUM_NODE_CLASSES,
        "hidden": best_params.get("hidden"),
        "heads": best_params.get("heads"),
        "layers": best_params.get("layers"),
        "dropout": best_params.get("dropout"),
    },
}

metadata_path = checkpoints_dir / metadata_filename
with open(metadata_path, "w") as f:
    json.dump(metadata, f, indent=2)
print(f"Saved metadata to: {metadata_path}")

print(f"\nModel saved successfully!")
print(f"  Model: {model_path}")
print(f"  Metadata: {metadata_path}")
